Em primeira instãncia , realizamos a instalação das bibliotecas:

In [1]:
!pip install fasttext-wheel


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import fasttext

In [3]:
import csv

In [4]:
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np

import gzip
import json

Abertura do arquivo:

In [6]:
file_path = "aotm2011_playlists.json.gz"
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    data = json.load(f)

Nosso DataSet das playlists em formato de DataFrame, e sem alterações prévias:

In [7]:
playlists = pd.read_json(file_path)
playlists

,category,filtered_lists,playlist,timestamp,mix_id,user
0,Mixed Genre,"[[SOFDPDC12A58A7D198], [SOPIEQP12A8C13F268, SO...","[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,89567,"{'mixes_posted': '23', 'member_since': '2004-0..."
1,Theme,"[[SOQOVCB12A58A7E83B], [SOGKLYN12AB017F155, SO...","[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,117124,"{'mixes_posted': '7', 'member_since': '2006-11..."
2,Theme,[[SOFIQBR12A6D4F78C6]],"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,97418,"{'mixes_posted': '93', 'member_since': '2003-1..."
3,Theme,"[[SORJUFZ12A67ADAF25, SOEELNN12AB018AC8D], [SO...","[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,23876,"{'mixes_posted': '8', 'member_since': '2002-03..."
4,Theme,"[[SOOUYTP12B0B8094B6, SOAJODS12A8C138A23], [SO...","[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,70694,"{'mixes_posted': '60', 'member_since': '2004-0..."
...,...,...,...,...,...,...
101338,Hip Hop,"[[SODLSVK12A8C13EEE1], [SOEOMEJ12A8AE46299], [...","[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,102561,"{'mixes_posted': '24', 'member_since': '2003-0..."
101339,Mixed Genre,"[[SONYEIJ12A6D4FA32D, SOSGUDA12A6D4FA2FB], [SO...","[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,69379,"{'mixes_posted': '1', 'member_since': '2004-03..."
101340,Single Artist,[[SOYOFST12D0219B4BA]],"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,18933,"{'mixes_posted': '23', 'member_since': '2001-1..."
101341,Theme,"[[SOKAWYU12A6D4F6769, SOXMBFV12A8C13B8DD], [SO...","[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,100251,"{'mixes_posted': '40', 'member_since': '2002-0..."


Pegaremos apenas as colunas que nos parecem interessantes:

In [8]:
playlists = playlists.drop(columns = ['user', 'mix_id', 'filtered_lists'])

In [9]:
playlists

,category,playlist,timestamp
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00
...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00


Apartir de agora, iremos testar um modelo para remover alguns caracteres indesejados da coluna playlist, como os colchetes por exemplo.

In [10]:
song_list = (playlists.iloc[1]['playlist'])
cleaned_list = [f"{band}::{song}" for (band, song), _ in song_list]
cleaned_list = ';'.join(cleaned_list)
print(cleaned_list)

The Cure::Edge Of the Deep Green Sea;Jeff Buckley::Nightmares By the Sea;Travis::Driftwood;Johnny Cash::Big River;G.Love::Fishing Song;REM::Nightswimming;The Doves::Caught By the River;Sugar::A Good Idea;Badly Drawn Boy::River, Sea, Ocean;Apples In Stereo::Stream Running Over;Frank Black & The Catholics::St. Francis Damn Disaster;The Tragically Hip::New Orleans Is Sinking;Tom Waits::Rain;Morphine::Wishing Well;They Might Be Giants::Cowtown;Talking Heads::Take Me To the River;Mike Doughty::Down On the River By the Sugar Plant;Porno For Pyros::Tahitian Moon;James::Sometimes


Removemos os espaços também

In [11]:
cleaned_list = cleaned_list.replace(' ', '')
cleaned_list

'TheCure::EdgeOftheDeepGreenSea;JeffBuckley::NightmaresBytheSea;Travis::Driftwood;JohnnyCash::BigRiver;G.Love::FishingSong;REM::Nightswimming;TheDoves::CaughtBytheRiver;Sugar::AGoodIdea;BadlyDrawnBoy::River,Sea,Ocean;ApplesInStereo::StreamRunningOver;FrankBlack&TheCatholics::St.FrancisDamnDisaster;TheTragicallyHip::NewOrleansIsSinking;TomWaits::Rain;Morphine::WishingWell;TheyMightBeGiants::Cowtown;TalkingHeads::TakeMeTotheRiver;MikeDoughty::DownOntheRiverBytheSugarPlant;PornoForPyros::TahitianMoon;James::Sometimes'

Fazemos então uma função que agrupa os passos anteriores: remove os colchetes, retira os espaços e faz devidas alterações na capitalização da palavra(MAIÚSCULA Ou minúscula)

In [12]:
def preprocess_playlist(song_list):
    # Função interna para capitalizar as palavras
    def capitalize_words(text):
        return ''.join(word.capitalize() for word in text.split())

    # Função interna para remover caracteres especiais
    def remove_special_chars(text):
        special_chars = "!@#$%^&*()_+=-{}[]|\:;\"'<>?,./"
        for char in special_chars:
            text = text.replace(char, "")
        return text

    # Processa cada (banda, música) da lista e limpa
    cleaned_list = []
    for (band, song), _ in song_list:
        # Capitaliza e remove caracteres especiais
        band = remove_special_chars(capitalize_words(band))
        song = remove_special_chars(capitalize_words(song))
        # Adiciona à lista formatada
        cleaned_list.append(f"{band}::{song}")

    # Junta todos os elementos em uma string separada por espaço
    cleaned_string = ' '.join(cleaned_list)
    
    return cleaned_string


playlist_limpa = playlists.get('playlist').apply(preprocess_playlist)
clean_playlist = playlists.assign(
    playlists_limpas = playlist_limpa
)
clean_playlist


<>:8: SyntaxWarning: invalid escape sequence '\:'
<>:8: SyntaxWarning: invalid escape sequence '\:'
C:\Users\ccaio\AppData\Local\Temp\ipykernel_9736\143419030.py:8: SyntaxWarning: invalid escape sequence '\:'
  special_chars = "!@#$%^&*()_+=-{}[]|\:;\"'<>?,./"


,category,playlist,timestamp,playlists_limpas
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrichsFavouritePoem The...
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea JeffBuckley::Ni...
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins OfraHazaAndIggy...
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights TheZombies::Tim...
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::ItsAWonderfulLife Whiskeytown::I...
...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,RKelly::BurnItUpremixFeatDaddyYankee 50Cent::C...
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave Prefuse73::AfternoonLoveIn Ni...
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody ::TieYourMotherDown ::...
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Bigjesustrashcan XmalDeutschlan...


In [13]:


# Função para contar o número de músicas por playlist
def count_tracks(playlist):
    return len(playlist.split())

# Contar o número de músicas em cada playlist
clean_playlist['num_musicas'] = clean_playlist['playlists_limpas'].apply(count_tracks)

# Calcular Q1 (primeiro quartil), Q3 (terceiro quartil) e IQR
Q1 = clean_playlist['num_musicas'].quantile(0.25)
Q3 = clean_playlist['num_musicas'].quantile(0.75)
IQR = Q3 - Q1

# Definir os limites
limite_inferior = 4  # Menos de 4 músicas é considerado curto
limite_superior = Q3 + 1.5 * IQR  # Playlists mais longas que esse valor são consideradas longas

# Filtrar as playlists que estão dentro do limite
clean_playlist = clean_playlist[(clean_playlist['num_musicas'] >= limite_inferior) & 
                                     (clean_playlist['num_musicas'] <= limite_superior)]

# Exibir o resultado
clean_playlist


,category,playlist,timestamp,playlists_limpas,num_musicas
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrichsFavouritePoem The...,14
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea JeffBuckley::Ni...,19
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins OfraHazaAndIggy...,16
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights TheZombies::Tim...,32
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::ItsAWonderfulLife Whiskeytown::I...,25
...,...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,RKelly::BurnItUpremixFeatDaddyYankee 50Cent::C...,30
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave Prefuse73::AfternoonLoveIn Ni...,17
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody ::TieYourMotherDown ::...,20
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Bigjesustrashcan XmalDeutschlan...,18


Criamos uma nova coluna, chamada 'new_category' que será o equivalente a 'category', porém, sem espaçamentos entre os termos. Mais adiante ficará mais claro o porquê disso.

In [14]:
import re

In [15]:
def processador(coluna):
    nova_coluna = re.sub(' ','',coluna)
    return nova_coluna

In [16]:
clean_playlist['new_category'] = clean_playlist['category'].map(processador).str.lower()
clean_playlist

C:\Users\ccaio\AppData\Local\Temp\ipykernel_9736\799961831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_playlist['new_category'] = clean_playlist['category'].map(processador).str.lower()


,category,playlist,timestamp,playlists_limpas,num_musicas,new_category
0,Mixed Genre,"[[[peter murphy, marlene dietrich's favourite ...",2005-03-27 10:53:00,PeterMurphy::MarleneDietrichsFavouritePoem The...,14,mixedgenre
1,Theme,"[[[The Cure, Edge Of the Deep Green Sea], None...",2008-01-11 21:45:00,TheCure::EdgeOfTheDeepGreenSea JeffBuckley::Ni...,19,theme
2,Theme,"[[[Peter Gabriel, The Feeling Begins], None], ...",2005-10-31 10:15:00,PeterGabriel::TheFeelingBegins OfraHazaAndIggy...,16,theme
3,Theme,"[[[The Animals, San Francisco Nights], None], ...",2002-03-20 17:59:00,TheAnimals::SanFranciscoNights TheZombies::Tim...,32,theme
4,Theme,"[[[Sparklehorse, It's a Wonderful Life], SOOUY...",2004-03-23 03:43:00,Sparklehorse::ItsAWonderfulLife Whiskeytown::I...,25,theme
...,...,...,...,...,...,...
101338,Hip Hop,"[[[R. Kelly, Burn It Up (Remix feat. Daddy Yan...",2006-04-03 14:54:00,RKelly::BurnItUpremixFeatDaddyYankee 50Cent::C...,30,hiphop
101339,Mixed Genre,"[[[Pachelbel, Grave], None], [[Prefuse73, Afte...",2004-03-03 15:05:00,Pachelbel::Grave Prefuse73::AfternoonLoveIn Ni...,17,mixedgenre
101340,Single Artist,"[[[Queen, Bohemian Rhapsody], SOYOFST12D0219B4...",2001-12-18 00:01:00,Queen::BohemianRhapsody ::TieYourMotherDown ::...,20,singleartist
101341,Theme,"[[[birthday party, big-jesus-trash-can], None]...",2006-01-28 10:16:00,BirthdayParty::Bigjesustrashcan XmalDeutschlan...,18,theme


Novamente, organizamos as colunas e filtramos apenas a que iremos usar:

In [17]:
clean_playlist = clean_playlist[['new_category','playlists_limpas', 'num_musicas']]
clean_playlist

,new_category,playlists_limpas,num_musicas
0,mixedgenre,PeterMurphy::MarleneDietrichsFavouritePoem The...,14
1,theme,TheCure::EdgeOfTheDeepGreenSea JeffBuckley::Ni...,19
2,theme,PeterGabriel::TheFeelingBegins OfraHazaAndIggy...,16
3,theme,TheAnimals::SanFranciscoNights TheZombies::Tim...,32
4,theme,Sparklehorse::ItsAWonderfulLife Whiskeytown::I...,25
...,...,...,...
101338,hiphop,RKelly::BurnItUpremixFeatDaddyYankee 50Cent::C...,30
101339,mixedgenre,Pachelbel::Grave Prefuse73::AfternoonLoveIn Ni...,17
101340,singleartist,Queen::BohemianRhapsody ::TieYourMotherDown ::...,20
101341,theme,BirthdayParty::Bigjesustrashcan XmalDeutschlan...,18


Aqui , para deixar claro algumas informações, foi usado groupby para sabermos a relação da quantidade de playlist(já limpas) com as categorias, o quê inclusive nos leva a observar e descobrir quais são as categorias do DataSet

In [18]:
clean_playlist.groupby('new_category').count().sort_values(ascending=False , by='num_musicas')

,playlists_limpas,num_musicas
new_category,,
mixedgenre,45084,45084
theme,14221,14221
singleartist,5678,5678
rock/pop,5191,5191
alternatingdj,4864,4864
indierock,4709,4709
romantic,2639,2639
roadtrip,1928,1928
hiphop,1453,1453


Tomamos então uma ação importante: 
Criaremos uma nova coluna chamada 'playlist_limpas_indicadas', e junto disso , adicionaremos o termo "__ label __" seguido de um espaço e o nome da categoria pertecente a essa playlist. Isso será muito importante para algumas funções usando a bilioteca 'fasttext'.

In [19]:
clean_playlist['playlists_limpas_indicadas'] = "__label__"+clean_playlist['new_category']+" "+clean_playlist['playlists_limpas']
clean_playlist = clean_playlist[['new_category','playlists_limpas_indicadas', 'playlists_limpas','num_musicas']]
clean_playlist

C:\Users\ccaio\AppData\Local\Temp\ipykernel_9736\3473419055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_playlist['playlists_limpas_indicadas'] = "__label__"+clean_playlist['new_category']+" "+clean_playlist['playlists_limpas']


,new_category,playlists_limpas_indicadas,playlists_limpas,num_musicas
0,mixedgenre,__label__mixedgenre PeterMurphy::MarleneDietri...,PeterMurphy::MarleneDietrichsFavouritePoem The...,14
1,theme,__label__theme TheCure::EdgeOfTheDeepGreenSea ...,TheCure::EdgeOfTheDeepGreenSea JeffBuckley::Ni...,19
2,theme,__label__theme PeterGabriel::TheFeelingBegins ...,PeterGabriel::TheFeelingBegins OfraHazaAndIggy...,16
3,theme,__label__theme TheAnimals::SanFranciscoNights ...,TheAnimals::SanFranciscoNights TheZombies::Tim...,32
4,theme,__label__theme Sparklehorse::ItsAWonderfulLife...,Sparklehorse::ItsAWonderfulLife Whiskeytown::I...,25
...,...,...,...,...
101338,hiphop,__label__hiphop RKelly::BurnItUpremixFeatDaddy...,RKelly::BurnItUpremixFeatDaddyYankee 50Cent::C...,30
101339,mixedgenre,__label__mixedgenre Pachelbel::Grave Prefuse73...,Pachelbel::Grave Prefuse73::AfternoonLoveIn Ni...,17
101340,singleartist,__label__singleartist Queen::BohemianRhapsody ...,Queen::BohemianRhapsody ::TieYourMotherDown ::...,20
101341,theme,__label__theme BirthdayParty::Bigjesustrashcan...,BirthdayParty::Bigjesustrashcan XmalDeutschlan...,18


Baixaremos  e importaremos uma importante biblioteca, que nos auxiliará na construção de arquivos vindo do nosso DataSet

In [20]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from sklearn.model_selection import train_test_split


Definiremos então tamanhos e proporções para os nossos arquivos de teste

In [22]:
train,test = train_test_split(clean_playlist,test_size = 0.15)

In [23]:
display(train.shape)
display(test.shape)

(83413, 4)

(14720, 4)

Nesse momento, usando os seguintes passos, atribuiremos a um arquivo (esse passo pode ser executado criando um novo arquivo) linhas que contém, uma por uma, as informações de cada playlist. Atenta-se ao fato, de que , caso seja rodado em outro dispositivo, realizar as devidas alterações quanto ao nome do arquivo

In [24]:
train.to_csv(r'train.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)
test.to_csv(r'teste.csv',columns=['playlists_limpas_indicadas'],index=False,header=False)

Treinaremos então um modelo , usando funções provinientes da biblioteca 'fasttext' e atribuindo a variável 'modelo'

In [25]:
modelo = fasttext.train_supervised(input="train.csv")

Usaremos o nosso o nosso arquivo de teste nesse momento

In [26]:
modelo.test(r"teste.csv")

(14720, 0.42527173913043476, 0.42527173913043476)

Aqui usaremos a função .predict da biblioteca fasttext para prever a classe de um texto fornecido, com base no modelo treinado.

In [27]:
modelo.predict(["Metallica::NothingElseMatters"],k=4)

([['__label__theme',
   '__label__singleartist',
   '__label__mixedgenre',
   '__label__alternatingdj']],
 [array([0.9684185 , 0.01228926, 0.00646777, 0.00572563], dtype=float32)])

Gensim

In [28]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from gensim.models.word2vec import LineSentence

# Agora use o LineSentence para carregar o arquivo de texto
sentences = LineSentence('train.txt')


In [30]:
# Exporta a coluna para um arquivo .txt, separando cada playlist por uma linha
train['playlists_limpas'].to_csv('train.txt', index=False, header=False, sep=' ')


In [31]:
from gensim.models import Word2Vec


In [32]:
gensim_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [33]:
# Configuração do modelo
gensim_model = Word2Vec(
    sentences,
    vector_size=100,  # Tamanho do vetor
    window=5,         # Janela de contexto
    min_count=5,      # Palavras raras serão ignoradas
    workers=6,        # Número de threads
    sg=1,             # Usar Skip-Gram
    epochs=10         # Número de épocas
)



In [34]:
gensim_model.save("word2vec_model.model")


In [35]:
# Encontrar palavras semelhantes a 'artista1'
gensim_model.wv.most_similar('Eminem::LoseYourself')


[('ChristinaAguilera::Dirrty', 0.9760281443595886),
 ('Eminem::SingForTheMoment', 0.9714016318321228),
 ('GoodCharlotte::TheAnthem', 0.9687115550041199),
 ('WillaFord::IWannaBeBad', 0.9685307145118713),
 ('Eminem::Superman', 0.9683312177658081),
 ('Ludacris::MoveBitch', 0.9666697382926941),
 ('BritneySpears::Stronger', 0.9660856127738953),
 ('LinkinPark::Faint', 0.9652292728424072),
 ('JustinTimberlake::LikeILoveYou', 0.9646934866905212),
 ('RKelly::Ignitionremix', 0.9646225571632385)]

*MODELO NAO SUPERVISIONADO*

In [36]:
train.to_csv(r'untrain.csv',columns=['playlists_limpas'],index=False,header=False)
test.to_csv(r'unteste.csv',columns=['playlists_limpas'],index=False,header=False)

In [37]:
modelo2 = fasttext.train_unsupervised(input = "untrain.csv")

In [38]:
modelo2.test(r"unteste.csv")

(0, nan, nan)

In [39]:
modelo2.get_nearest_neighbors("B2k::BumpBumpBump")

[(0.9504131078720093, 'B2k::UhHuh'),
 (0.9421402812004089, 'JaRuleFtAshanti::Mesmerize'),
 (0.9310137629508972, 'LilKim::JumpOff'),
 (0.9261728525161743, 'SeanPaul::GetBusy'),
 (0.9177154898643494, 'SeanPaul::LikeGlue'),
 (0.9120768904685974, 'PeteyPablo::Freekaleek'),
 (0.9093179106712341, 'Dmx::XGonGiveItToYa'),
 (0.9089449644088745, 'JayZ::BigPimpin'),
 (0.906396746635437, 'Choppa::ChoppaStyle'),
 (0.9062119722366333, 'SnoopDogg::DropItLikeItsHot')]

In [40]:
modelo2.get_nearest_neighbors("LinkinPark")

[(0.9282870292663574, 'LinkinPark::Papercut'),
 (0.9136923551559448, 'LinkinPark::ByMyself'),
 (0.9136862754821777, 'LinkinPark::Crawling'),
 (0.9131993055343628, 'LinkinPark::Runaway'),
 (0.9076439738273621, 'LinkinPark::OneStepCloser'),
 (0.9071182012557983, 'LinkinPark::Numb'),
 (0.9021285772323608, 'LinkinPark::Faint'),
 (0.9019230008125305, 'LinkinPark::EasierToRun'),
 (0.9012300372123718, 'LinkinPark::PointsOfAuthority'),
 (0.8933952450752258, 'LinkinPark::PushingMeAway')]

In [41]:
modelo2.get_nearest_neighbors("PearlJam::Black LinkinPark::Numb ")

[(0.9000174403190613, 'PearlJam::Black'),
 (0.8759235739707947, 'PearlJam::Blood'),
 (0.8749591708183289, 'PearlJam::HardToImagine'),
 (0.8661639094352722, 'PearlJam::Garden'),
 (0.865385890007019, 'PearlJam::Evenflow'),
 (0.8638047575950623, 'PearlJam::BrainOfJ'),
 (0.8590883612632751, 'PearlJam::Immortality'),
 (0.858096718788147, 'PearlJam::SpinTheBlackCircle'),
 (0.8571552038192749, 'PearlJam::Jeremy'),
 (0.8564835786819458, 'PearlJam::GivenToFly')]

In [42]:
modelo2.get_word_vector("Metallica::NothingElseMatters")

array([-0.77865446, -0.29081503, -0.67350906, -0.6944459 ,  0.05970585,
       -0.329549  , -0.5451959 , -0.09038128,  0.36404815, -0.01864655,
       -0.906044  ,  0.50127554,  0.6249414 ,  0.32641628, -0.1951039 ,
        0.03699486, -0.60675114,  0.72843915,  0.1272802 , -0.2654684 ,
        0.12211129,  0.2515792 , -0.34735706, -0.51726127,  0.708254  ,
       -0.6196049 ,  0.06218853,  0.18644238, -0.62388706,  0.33194432,
       -0.81134707,  0.35663962,  0.25329873, -1.077862  , -0.22395693,
        0.23257938,  0.65882736, -0.30912164, -0.08434164, -0.47947317,
       -0.3131586 ,  0.0538157 , -0.04110965,  0.07616454, -0.36676016,
        0.37883073, -0.49127018,  0.16327631,  0.19410583,  0.4664081 ,
       -0.06568651,  0.3409685 , -0.6752339 ,  0.30174267, -0.3959357 ,
        0.2743549 , -0.6141972 , -0.05243463,  0.30863026,  0.5053125 ,
        0.8115573 , -0.16571549,  0.83227134,  0.2929229 ,  0.11095376,
       -0.03469239, -0.06693441,  1.1895801 , -0.01945785,  1.39

In [43]:
modelo2.get_word_vector("Metallica::One")

array([-0.88637996, -0.17533907, -0.02370962, -1.1732366 ,  0.23563324,
       -0.5221716 , -0.5025147 , -0.2827829 ,  0.15782323,  0.10741871,
       -0.72997445, -0.00430872,  0.06871902,  1.0668255 , -0.5099471 ,
        0.36522096, -0.5683457 ,  0.8776179 ,  0.4335196 , -0.922605  ,
       -0.03213974,  0.16507411, -0.11288075, -0.3809798 ,  1.0785266 ,
       -1.6064162 , -0.27294347, -0.14758231, -0.49035525,  0.16789773,
       -1.5516741 ,  0.31097448,  0.63301134, -0.93889993, -0.8438755 ,
        0.1724316 ,  1.0746905 , -0.06906839, -0.29857033, -1.1392813 ,
       -0.18351965, -0.75167716, -0.2929186 ,  0.3901813 ,  0.20114994,
        1.1980207 , -0.6089859 ,  0.33714625,  0.33828476,  1.3423904 ,
        0.56566274,  0.6678814 , -0.8262787 ,  0.557954  , -0.41065234,
        0.19120146, -0.26223755,  0.11063916,  0.39389345,  0.10041324,
        0.7816267 , -0.35112137,  0.8806635 ,  1.3368287 , -0.01733886,
        0.2732296 , -0.45571077,  1.1916558 , -0.46162218,  1.88

In [44]:
modelo2.get_words(include_freq=True)

(['</s>',
  '::',
  'NickDrake::PinkMoon',
  'MazzyStar::FadeIntoYou',
  'JoyDivision::LoveWillTearUsApart',
  'TheSmiths::Asleep',
  'NeutralMilkHotel::InTheAeroplaneOverTheSea',
  'TheCure::JustLikeHeaven',
  'TheShins::NewSlang',
  'ThePostalService::SuchGreatHeights',
  'NeutralMilkHotel::Holland1945',
  'JeffBuckley::Hallelujah',
  'GaryJules::MadWorld',
  'Air::PlaygroundLove',
  'Nico::TheseDays',
  'Radiohead::FakePlasticTrees',
  'Radiohead::Creep',
  'Coldplay::Yellow',
  'Coldplay::TheScientist',
  'JeffBuckley::LastGoodbye',
  'YeahYeahYeahs::Maps',
  'Weezer::ElScorcho',
  'TheSmiths::ThereIsALightThatNeverGoesOut',
  'PinkFloyd::WishYouWereHere',
  'Oasis::Wonderwall',
  'ViolentFemmes::BlisterInTheSun',
  'Wilco::JesusEtc',
  'BrightEyes::LoverIDontHaveToLove',
  'Pixies::WhereIsMyMind',
  'LouReed::PerfectDay',
  'FooFighters::Everlong',
  'ThePixies::WhereIsMyMind',
  'Spoon::TheWayWeGetBy',
  'Weezer::IslandInTheSun',
  'Radiohead::KarmaPolice',
  'DeathCabForCutie::P

In [45]:
modelo2.get_nearest_neighbors("Eminem::LoseYourself")

[(0.8809277415275574, 'Eminem::GetYouMad'),
 (0.8607431054115295, 'Eminem::IfIHad'),
 (0.8515639901161194, 'Eminem::KillYou'),
 (0.8459057211875916, 'Eminem::8Mile'),
 (0.8419108390808105, 'Eminem::Kim'),
 (0.8410542607307434, 'Eminem::MyFault'),
 (0.8332277536392212, 'Eminem::RabbitRun'),
 (0.8285439014434814, 'Eminem::RunRabbitRun'),
 (0.8234468698501587, 'Eminem::AssLikeThat'),
 (0.8212878704071045, 'Eminem::8MileRoad')]

In [46]:
modelo2.get_nearest_neighbors("Beatles::YellowSubmarine RollingStones::PaintItBlack LedZeppelin::StairwayToHeaven PinkFloyd::WishYouWereHere Queen::BohemianRhapsody TheDoors::LightMyFire TheWho::BabaO'Riley Nirvana::SmellsLikeTeenSpirit ACDC::BackInBlack U2::WithOrWithoutYou")

[(0.7576283812522888, 'Meatloaf::BatOutOfHell'),
 (0.7509943246841431, 'Steppenwolf::BornToBeWild'),
 (0.7410396933555603, 'Who::PinballWizard'),
 (0.7338366508483887, 'GunsnRoses::KnockinOnHeavensDoor'),
 (0.7318937182426453, 'Who::WontGetFooledAgain'),
 (0.7254345417022705, 'GunsNRoses::KnockinOnHeavensDoor'),
 (0.7250596284866333, 'LedZepplin::StairwayToHeaven'),
 (0.722780704498291, 'Scorpions::RockYouLikeAHurricane'),
 (0.7216320037841797, 'Kiss::GodGaveRockAndRollToYou'),
 (0.7201417684555054, 'LynardSkynard::Freebird')]

In [47]:
modelo2.get_nearest_neighbors("Eminem::LoseYourself JayZ::99Problems KanyeWest::GoldDigger 50Cent::InDaClub MissyElliott::GetUrFreakOn OutKast::HeyYa Nelly::HotInHerre Ludacris::StandUp LilWayne::GoDJ SnoopDogg::DropItLikeItsHot")

[(0.9143017530441284, 'SeanPaul::GetBusy'),
 (0.911916196346283, 'SnoopDogg::DropItLikeItsHot'),
 (0.9081235527992249, 'LilKim::JumpOff'),
 (0.9010723829269409, 'B2k::BumpBumpBump'),
 (0.8958092927932739, 'Ludacris::RolloutmyBusiness'),
 (0.892342746257782, 'Dmx::XGonGiveItToYa'),
 (0.8920081257820129, 'SnoopDogg::LayLow'),
 (0.8897716403007507, 'Xzibit::Multiply'),
 (0.887927234172821, 'Xzibit::X'),
 (0.887448787689209, 'JaRuleFtAshanti::Mesmerize')]

In [48]:
modelo2.get_nearest_neighbors("Eminem::LoseYourself JayZ::99Problems KanyeWest::GoldDigger 50Cent::InDaClub MissyElliott::GetUrFreakOn LedZeppelin::StairwayToHeaven PinkFloyd::WishYouWereHere Queen::BohemianRhapsody TheDoors::LightMyFire TheWho::BabaO'Riley Nirvana::SmellsLikeTeenSpirit")

[(0.7727984189987183, 'Foghat::Slowride'),
 (0.7683150768280029, 'TedNugent::FreeForAll'),
 (0.7651644945144653, 'Acdc::WhoMadeWho'),
 (0.75876784324646, 'QuietRiot::BangYourHeadmetalHealth'),
 (0.7567671537399292, 'Meatloaf::BatOutOfHell'),
 (0.7529360055923462, 'Acdc::ForThoseAboutToRockweSaluteYou'),
 (0.7510794997215271, 'DefLeppard::ArmageddonIt'),
 (0.7510153651237488, 'D12::PurpleHills'),
 (0.7509395480155945, 'Kiss::GodGaveRockAndRollToYou'),
 (0.7444747090339661, 'LynardSkynard::Freebird')]

*TENTATIVA PLOTAR VETORES PARA COMPARAR*
